In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
import json
from proq_gen.generators.chroma import get_db_store
from proq_gen.generators.ideation import get_ideation_chain
from proq_gen.generators.generator import get_generator_chain
from proq_gen.generators.test_case import update_data_store_with_test_cases


with open("data/python_kb.json") as f:
    python_kb = json.load(f)

ideation_chain = get_ideation_chain(
    lang="python",
    concept_groups=[
        (
            python_kb["data_processing"]["concepts"],
            6,
        ),  # randomly picks 6 concepts from all concepts
        (
            python_kb["data_formats"],
            5,
        ),  # randomly picks 5 data formats from all data formats
    ],
    **python_kb["data_processing"]["ideation"]
)



db_store = get_db_store("python-questions")

generator_chain = get_generator_chain(ideation_chain, db_store)

/home/saranath/ai-proq-gen/myvenv/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# generator_chain = get_generator_chain(ideation_chain, db_store)
generator_chain.invoke({"n_problems": 5})

data_store = update_data_store_with_test_cases(db_store)
print(data_store)

AttributeError: 'Chroma' object has no attribute 'copy'

In [28]:
doc = db_store.similarity_search("count")[0]

In [29]:
doc.metadata["testcaes"] = {"input":"print('hello')", "output":"hello"}

In [35]:
data = db_store.get()

In [39]:
data

{'ids': ['03c32fa3-de05-4149-8cc4-88773151f547',
  '09b573e4-c770-4b8d-b6c0-dc2d0acabf1c',
  '1653470e-6d06-48a9-8769-5e1240da7190',
  '17f50f95-c5e3-489a-8c68-263f00400dfa',
  '3c381cf2-b6b2-40e2-8d0c-cff7e578f3f9',
  '43f27f5e-12b3-4c14-9b2b-263ae0b78520',
  '4ea14eb8-8ea2-4729-97e9-4b1a1fd3b4b9',
  '56ab1e93-559c-4017-9da6-3b863eb5f202',
  '5844f0b5-fcd0-4490-9544-4d015583544d',
  '5d5863af-79b8-444b-aa3c-53a2af318acc',
  '66d68320-1264-480a-827d-8dc3ddc2ac7e',
  '764803c5-7251-4150-b735-ab2d46e99d66',
  '7c484fce-b222-478c-9fbf-092a2ce62ca9',
  '862b0c22-6ebd-48b7-8ef6-055d4dbd10a2',
  '8a1865f7-19ec-4168-a7d9-a8182693e96e',
  '8d180a75-c645-4d6f-9afa-949435d39be8',
  'b06c816c-fd72-43ef-b295-87878d747d41',
  'b2f95667-0335-4e3c-b48b-4c949d0b2974',
  'c0e3e032-172f-4ffa-93e6-36cc60ee5aae',
  'c0ff2e3e-62d3-48cd-86dc-3aa6396a2701',
  'c2d99bff-60ba-4dbc-993e-cbe1d5244d22',
  'c4df2c10-600d-4fdd-a53a-1f26ffef66cb',
  'cd0ee178-3ebf-4317-9fa9-6baf2980f4ed'],
 'embeddings': None,
 'met

In [42]:
id_docs  = [
    {'document_id':id,"document":Document(page_content=page_content, metadata=metadata)}
    for id, page_content, metadata in zip(data["ids"], data["documents"], data["metadatas"])
]


In [48]:
db_store.get(id_docs[0]["document_id"])

{'ids': ['03c32fa3-de05-4149-8cc4-88773151f547'],
 'embeddings': None,
 'metadatas': [{'data_formats': 'str,int',
   'solution': 'def count_consecutive_repeating_chars(s:str)->int:\n    count = 1\n    total_count = 0\n    for i in range(1, len(s)):\n        if s[i] == s[i-1]:\n            count += 1\n        else:\n            total_count += count\n            count = 1\n    return total_count + count',
   'tags': 'count,loop,string',
   'testcases': 'sdfsdf'}],
 'documents': ['Find the count of consecutive repeating characters in a string.'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [46]:
id_docs[0]["document"].metadata["testcases"] = "sdfsdf"

In [47]:
db_store.update_document(**id_docs[0])

In [24]:
from langchain_core.documents import Document

In [26]:
Document()

TypeError: Document.__init__() missing 1 required positional argument: 'page_content'

In [ ]:
db_store.similarity_search("matrix")

[Document(metadata={'data_formats': 'int,list[list]', 'solution': 'def multiply_by_two(matrix:list[list[int]])->list[list[int]]:\n    return [[num * 2 for num in row] for row in matrix]', 'tags': 'matrix,multiplication,list-comprehension', 'test_cases': '[\n    {"input": [[1, 2], [3, 4]], "output": [[2, 4], [6, 8]]},\n    {"input": [[-1, 0], [5, 10]], "output": [[-2, 0], [10, 20]]},\n    {"input": [[0, 0], [0, 0]], "output": [[0, 0], [0, 0]]}\n]'}, page_content='Given a matrix, return a new matrix with all numbers multiplied by 2'),
 Document(metadata={'data_formats': 'int,list[list]', 'solution': 'def count_even_rows(matrix:list[list[int]])->int:\n    return sum(1 for row in matrix if len(row) % 2 == 0)', 'tags': 'count,matrix,filtering', 'test_cases': '[\n    {"input": [[1, 2], [3, 4, 5], [6, 7, 8, 9]], "output": 1},\n    {"input": [[1, 2, 3, 4], [5, 6], [7, 8, 9, 10]], "output": 2},\n    {"input": [[1, 2, 3], [4, 5, 6], [7, 8, 9]], "output": 0}\n]'}, page_content='Find the number of

In [ ]:
db_store.similarity_search("a")

[Document(metadata={'data_formats': 'str,dict', 'solution': 'def word_count(sentence:str)->dict:\n    return {word: sentence.count(word) for word in sentence.split()}', 'tags': 'count,split,dictionary', 'test_cases': '[\n    {"input": "hello world", "output": {"hello": 1, "world": 1}},\n    {"input": "hello hello world world", "output": {"hello": 2, "world": 2}},\n    {"input": "", "output": {}}\n]'}, page_content='Given a string, return a dictionary with count of each word'),
 Document(metadata={'data_formats': 'str,list', 'solution': "def concat_strings(strings:list[str])->str:\n    return '-'.join(strings)", 'tags': 'concatenation,join', 'test_cases': '[\n    {"input": ["hello", "world", "python"], "output": "hello-world-python"},\n    {"input": ["a", "b", "c", "d"], "output": "a-b-c-d"},\n    {"input": ["single"], "output": "single"}\n]'}, page_content='Given a list of strings, return a string with all strings concatenated and separated by hyphens'),
 Document(metadata={'data_forma

In [ ]:
db_store.get_texts_by_ids(['764803c5-7251-4150-b735-ab2d46e99d66'])

AttributeError: 'Chroma' object has no attribute 'get_texts_by_ids'